In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
#np.set_printoptions(threshold=sys.maxsize)

In [2]:
data =  pd.read_csv('/content/drive/MyDrive/csh111.ann.features.csv')

In [3]:
data

,lastSensorEventHours,lastSensorEventSeconds,lastSensorDayOfWeek,windowDuration,timeSinceLastSensorEvent,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,complexity,activityChange,areaTransitions,numDistinctSensors,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea,activity
0,0.0,1258.0,2.0,698.0,1.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.028653,0.0,0.0,0.0,28.62,0.0,0.0,0.0,5.73,0.0,0.0,0.0,0.0,0.0,86400.000000,0.000000,86400.0,86400.0,86400.0,1.571419,86400.000000,86400.000000,86400.0,86400.000000,86400.000000,Sleep
1,0.0,1260.0,2.0,699.0,2.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.028612,0.0,0.0,0.0,28.57,0.0,0.0,0.0,5.78,0.0,0.0,0.0,0.0,0.0,86400.000000,0.000000,86400.0,86400.0,86400.0,2.852767,86400.000000,86400.000000,86400.0,86400.000000,86400.000000,Sleep
2,0.0,1282.0,2.0,720.0,22.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.029167,0.0,0.0,0.0,28.52,0.0,0.0,0.0,5.83,0.0,0.0,0.0,0.0,0.0,86400.000000,0.000000,86400.0,86400.0,86400.0,25.571251,86400.000000,86400.000000,86400.0,86400.000000,86400.000000,Sleep
3,0.0,1283.0,2.0,719.0,1.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.027816,0.0,0.0,0.0,28.47,0.0,0.0,0.0,5.88,0.0,0.0,0.0,0.0,0.0,86400.000000,0.000000,86400.0,86400.0,86400.0,26.665044,86400.000000,86400.000000,86400.0,86400.000000,86400.000000,Sleep
4,0.0,1329.0,2.0,756.0,46.0,0.0,0.0,1.0,1.0,1.0,0.650022,0.017196,0.0,0.0,0.0,28.42,0.0,0.0,0.0,5.93,0.0,0.0,0.0,0.0,0.0,86400.000000,0.000000,86400.0,86400.0,86400.0,72.605688,86400.000000,86400.000000,86400.0,86400.000000,86400.000000,Sleep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351319,23.0,84823.0,6.0,1966.0,1.0,1.0,1.0,1.0,1.0,1.0,0.047279,0.489827,0.0,0.0,0.0,33.33,0.0,0.0,0.0,1.02,0.0,0.0,0.0,0.0,0.0,6568.683782,0.000000,86400.0,86400.0,86400.0,1.106426,6568.263181,6564.643840,86400.0,6775.523950,6776.837340,Sleep
351320,23.0,84832.0,6.0,1973.0,9.0,1.0,1.0,5.0,5.0,1.0,0.353359,0.487582,0.0,0.0,0.0,32.32,0.0,0.0,0.0,2.03,0.0,0.0,0.0,0.0,0.0,6577.739691,9.055909,86400.0,86400.0,86400.0,0.000000,6577.319090,6573.699749,86400.0,6784.579859,6785.893249,Other_Activity
351321,23.0,84832.0,6.0,1972.0,0.0,1.0,1.0,1.0,1.0,1.0,0.353359,0.489858,0.0,0.0,0.0,32.30,0.0,0.0,0.0,2.05,0.0,0.0,0.0,0.0,0.0,6577.799570,0.000000,86400.0,86400.0,86400.0,0.059879,6577.378969,6573.759628,86400.0,6784.639738,6785.953128,Sleep
351322,23.0,84834.0,6.0,1973.0,2.0,1.0,1.0,1.0,1.0,1.0,0.353359,0.489103,0.0,0.0,0.0,32.28,0.0,0.0,0.0,2.07,0.0,0.0,0.0,0.0,0.0,6578.923727,0.000000,86400.0,86400.0,86400.0,1.184036,6578.503126,6574.883785,86400.0,6785.763895,6787.077285,Sleep


In [4]:
data.dtypes

lastSensorEventHours        float64
lastSensorEventSeconds      float64
lastSensorDayOfWeek         float64
windowDuration              float64
timeSinceLastSensorEvent    float64
prevDominantSensor1         float64
prevDominantSensor2         float64
lastSensorID                float64
lastSensorLocation          float64
lastMotionLocation          float64
complexity                  float64
activityChange              float64
areaTransitions             float64
numDistinctSensors          float64
sensorCount-Bathroom        float64
sensorCount-Bedroom         float64
sensorCount-Chair           float64
sensorCount-DiningRoom      float64
sensorCount-Hall            float64
sensorCount-Ignore          float64
sensorCount-Kitchen         float64
sensorCount-LivingRoom      float64
sensorCount-Office          float64
sensorCount-OutsideDoor     float64
sensorCount-WorkArea        float64
sensorElTime-Bathroom       float64
sensorElTime-Bedroom        float64
sensorElTime-Chair          

In [5]:
data_in = data.iloc[:,3:36].values
data_out = data.iloc[:,36].values

In [6]:
data_out

array(['Sleep', 'Sleep', 'Sleep', ..., 'Sleep', 'Sleep', 'Other_Activity'],
      dtype=object)

In [7]:
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
def scale(data):
  # define contrived series
  #series = Series(data)
  # prepare data for normalization
  #values = data.values
  values = data.reshape((len(data), 1))
  # train the normalization
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaler = scaler.fit(values)
  # normalize the dataset and print
  normalized = scaler.transform(values)
  return normalized

In [8]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
def encod(data):
  values = array(data)
  # integer encode
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(values)
  # binary encode
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
  return onehot_encoded

In [9]:
out = encod(data_out)

In [10]:
for col in range(0,33):
  scaler = scale(data_in[:,col])
  #stan = standard(scaler.flatten())
  data_in[:,col:col+1] = scaler

In [11]:
data_in.shape

(351324, 33)

In [12]:
X = []
y = []

In [13]:
for row in range(50,len(data_in[:,0])):
  X.append(data_in[row-50: row,:])
  y.append(out[row])

In [14]:
len(y)

351274

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
type(x_train)

list

In [17]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
type(y_train)

numpy.ndarray

In [ ]:
x_train.shape

(281019, 50, 33)

In [ ]:
y_train.shape

(281019, 35)

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0], x_train.shape[1], 33))

In [ ]:
#y_train = np.reshape(y_train,(y_train.shape[0], 35))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = Sequential()
#First layer
model.add(LSTM(units=50,return_sequences=True, input_shape=( x_train.shape[1],33) ))

#LSTM Second layer
model.add(LSTM(units=50, return_sequences=True))

#LSTM Third layer
model.add(LSTM(units=50))

#model.add(layers.Activation('softmax'))
#Dense layer
model.add(Dense(units=35, activation='softmax'))


In [ ]:
#compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#fit
model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)

Epoch 1/20
1757/1757 [==============================] - 31s 13ms/step - loss: 1.2530 - accuracy: 0.6281 - val_loss: 1.0012 - val_accuracy: 0.6793
Epoch 2/20
1757/1757 [==============================] - 21s 12ms/step - loss: 0.9567 - accuracy: 0.6894 - val_loss: 0.8986 - val_accuracy: 0.7025
Epoch 3/20
1757/1757 [==============================] - 21s 12ms/step - loss: 0.8730 - accuracy: 0.7091 - val_loss: 0.8746 - val_accuracy: 0.7088
Epoch 4/20
1757/1757 [==============================] - 23s 13ms/step - loss: 0.8139 - accuracy: 0.7237 - val_loss: 0.8050 - val_accuracy: 0.7278
Epoch 5/20
1757/1757 [==============================] - 21s 12ms/step - loss: 0.7606 - accuracy: 0.7382 - val_loss: 0.7374 - val_accuracy: 0.7447
Epoch 6/20
1757/1757 [==============================] - 21s 12ms/step - loss: 0.7212 - accuracy: 0.7496 - val_loss: 0.7159 - val_accuracy: 0.7524
Epoch 7/20
1757/1757 [==============================] - 22s 12ms/step - loss: 0.6787 - accuracy: 0.7627 - val_loss: 0.6775 -

In [24]:
x_test, y_test = np.array(x_test), np.array(y_test)

In [27]:
x_test = np.reshape(x_test,(x_test.shape[0], 5, 10, 33))

In [28]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

2196/2196 [==============================] - 13s 6ms/step - loss: 0.2013 - accuracy: 0.9420
Test Loss: 0.20130935311317444
Test Accuracy: 0.9419969916343689


In [18]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers import TimeDistributed

In [19]:
x_train = np.reshape(x_train,( x_train.shape[0], 5, 10, 33))

In [20]:
model = Sequential()
# after having Conv2D...
model.add(TimeDistributed(Conv1D(64, (2), activation='relu'), input_shape=( 5, 10, 33)))


model.add(TimeDistributed(Conv1D(64, (2), activation='relu')))


model.add(TimeDistributed(MaxPooling1D(pool_size=(2))))
# We need to have only one dimension per output
# to insert them to the LSTM layer - Flatten or use Pooling
model.add(TimeDistributed(Flatten()))
# previous layer gives 5 outputs, Keras will make the job
# to configure LSTM inputs shape (5, ...)
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
# and then, common Dense layers... Dropout...
# up to you
model.add(Dense(35, activation='softmax'))

model.compile('adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(x_train, y_train, batch_size=512, epochs=200, validation_split=0.2)

Epoch 1/200
440/440 [==============================] - 37s 47ms/step - loss: 1.6571 - accuracy: 0.5407 - val_loss: 1.2293 - val_accuracy: 0.6498
Epoch 2/200
440/440 [==============================] - 19s 42ms/step - loss: 1.1396 - accuracy: 0.6590 - val_loss: 1.0308 - val_accuracy: 0.6811
Epoch 3/200
440/440 [==============================] - 19s 42ms/step - loss: 1.0117 - accuracy: 0.6799 - val_loss: 0.9567 - val_accuracy: 0.6922
Epoch 4/200
440/440 [==============================] - 19s 43ms/step - loss: 0.9384 - accuracy: 0.6942 - val_loss: 0.8838 - val_accuracy: 0.7067
Epoch 5/200
440/440 [==============================] - 18s 42ms/step - loss: 0.8883 - accuracy: 0.7044 - val_loss: 0.8562 - val_accuracy: 0.7145
Epoch 6/200
440/440 [==============================] - 18s 42ms/step - loss: 0.8508 - accuracy: 0.7131 - val_loss: 0.8272 - val_accuracy: 0.7202
Epoch 7/200
440/440 [==============================] - 19s 42ms/step - loss: 0.8168 - accuracy: 0.7216 - val_loss: 0.7900 - val_ac